In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 
import networkx as nx 
import pickle as pkl
import datetime as dt

import h5py

pd.set_option('display.max_columns', 100)


In [2]:
# gene id mapping
with open('../../../GENEID_MAPPING/data/processed/all2genesymbol.pkl', 'rb') as f: 
    all2symb = pkl.load(f)

sampinfo = pd.read_csv('../data/raw/instinfo_beta.txt', '\t', dtype=str)

geneinfo = pd.read_csv('../data/raw/geneinfo_beta.txt', sep='\t', dtype=str)
id2symb = geneinfo.set_index('gene_id')['gene_symbol'].to_dict()

expr_info = pd.read_csv('../data/raw/instinfo_beta.txt', sep='\t')
expr_info.head()

C:\Users\natha\miniconda3\envs\gcnvae\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_itime,pert_time_unit,cell_mfc_name,pert_mfc_id,det_plate,det_well,rna_plate,rna_well,count_mean,count_cv,qc_f_logp,qc_iqr,qc_slope,pert_id,sample_id,pert_type,cell_iname,qc_pass,dyn_range,inv_level_10,build_name,failure_mode,project_code,cmap_name
0,b11,NaN,20.00,uL,20 uL,72.0,72 h,h,VCAP,ERG_11,ERG013_VCAP_72H_X3_B11,O14,ERG013_VCAP_72H_X3,O14,59,18,7.3,11.74,66,ERG_11,ERG013_VCAP_72H_X3_B11:O14,trt_sh,VCAP,0.0,4.20788,4220.5,NaN,dyn_range,ERG,ERG
1,b10,NaN,1.00,uL,1 uL,96.0,96 h,h,U2OS,TRCN0000072237,TAK004_U2OS_96H_X2_B10_DUO52HI53LO,D10,TAK004_U2OS_96H_X1,D10,67,18,5.7,14.98,67,TRCN0000072237,TAK004_U2OS_96H_X2_B10_DUO52HI53LO:D10,ctl_vector,U2OS,0.0,4.73906,1462.0,NaN,inv_level_10,TAK,LACZ
2,b12,NaN,0.10,ng/ml,0.1 ng/ml,2.0,2 h,h,HEPG2,SOD3,CYT001_HEPG2_2H_X2_B12,N12,CYT001_HEPG2_2H_X2,N12,60,17,7.3,11.45,65,SOD3,CYT001_HEPG2_2H_X2_B12:N12,trt_lig,HEPG2,1.0,6.79642,3038.0,NaN,NaN,CYT,SOD3
3,b12,NaN,150.00,ng,150 ng,48.0,48 h,h,HEK293T,ENTRY00543,HSF038_HEK293T_48H_X2_B12,M01,HSF038_HEK293T_48H_X2,M01,75,15,5.9,9.25,59,ENTRY00543,HSF038_HEK293T_48H_X2_B12:M01,trt_oe,HEK293T,0.0,23.79710,1642.0,NaN,inv_level_10,HSF,PDGFRA
4,f3b5,6.66,5.33,uM,6.66 uM,24.0,24 h,h,A375,BRD-K79781870,DOS043_A375_24H_X1_F3B5_DUO52HI53LO,D17,DOS043_A375_24H_X1,D17,58,26,6.3,15.87,68,BRD-K79781870,DOS043_A375_24H_X1_F3B5_DUO52HI53LO:D17,trt_cp,A375,0.0,6.78867,1558.0,NaN,"inv_level_10,qc_iqr",DOS,BRD-K79781870


In [3]:
with open('../../../cell_baseline.pkl', 'rb') as f: 
    baseline_expr = pkl.load(f)
    
print('keys:', baseline_expr.keys())

cell_lines = baseline_expr['linc_ccle_lines']
len(cell_lines)

keys: dict_keys(['unpert_expr', 'linc_ccle_lines', 'meta', 'source', 'creation_date'])


138

In [4]:
with open('../../../DDS_graph.pkl', 'rb') as f: 
    DDS_graph = pkl.load(f)

with open('../../../PPI_graph.pkl', 'rb') as f: 
    PPI_graph = pkl.load(f)

with open('../../../cell_baseline.pkl', 'rb') as f: 
    cell_baseline = pkl.load(f)

lines = cell_baseline['linc_ccle_lines'].cell_iname.unique()
drugs = DDS_graph['nodelist']
genes = PPI_graph['nodelist']

print('# cell lines:\t', len(lines))
print('# drugs:     \t', len(drugs))
print('# genes:     \t', len(genes))

# cell lines:	 138
# drugs:     	 1224
# genes:     	 13565


In [5]:
cp = h5py.File('../data/raw/level3_beta_trt_cp_n1805898x12328.gctx', 'r')

h5_data= cp['0']['DATA']['0']['matrix']

sample_meta = np.array(cp['0']['META']['COL']['id'], dtype=str)
gene_meta = np.array(cp['0']['META']['ROW']['id'], dtype=str)

print('sample meta:', sample_meta[0:5])
print('gene meta:', gene_meta[0:5])
print(h5_data.shape)

sample meta: ['ABY001_A375_XH_X1_B15:A13' 'ABY001_A375_XH_X1_B15:A14'
 'ABY001_A375_XH_X1_B15:A15' 'ABY001_A375_XH_X1_B15:A16'
 'ABY001_A375_XH_X1_B15:A17']
gene meta: ['10' '100' '1000' '10000' '10001']
(1805898, 12328)


# Step 1: Select `Best Inferred` Genes and intersect with `PPI_graph` gene nodes. 

Use this to make the gene selector. 

In [6]:
# select subset of genes we want to train with
geneinfo2 = geneinfo[lambda x: x.feature_space.isin(['best inferred', 'landmark'])]

# we want to focus on protein coding genes - select subset
geneinfo2 = geneinfo2[lambda x: x.gene_type == 'protein-coding']

# make sure we're using the same gene id's 
geneinfo2 = geneinfo2.assign(myid = lambda x: x.gene_symbol.str.upper().map(PPI_graph['genemap']))

# filter any genes that don't have a gene id mapping 
geneinfo2 = geneinfo2[lambda x: ~x.myid.isna()]

# filter to genes in PPI graph 
geneinfo2 = geneinfo2[lambda x: x.myid.isin(PPI_graph['nodelist'])]

print('shape:', geneinfo2.shape)
geneinfo2.head()

shape: (8240, 8)


,gene_id,gene_symbol,ensembl_id,gene_title,gene_type,src,feature_space,myid
2155,23,ABCF1,ENSG00000204574,ATP binding cassette subfamily F member 1,protein-coding,NCBI,landmark,ABCF1
2156,25,ABL1,ENSG00000097007,"ABL proto-oncogene 1, non-receptor tyrosine ki...",protein-coding,NCBI,landmark,ABL1
2157,30,ACAA1,ENSG00000060971,acetyl-CoA acyltransferase 1,protein-coding,NCBI,landmark,ACAA1
2158,39,ACAT2,ENSG00000120437,acetyl-CoA acetyltransferase 2,protein-coding,NCBI,landmark,ACAT2
2159,47,ACLY,ENSG00000131473,ATP citrate lyase,protein-coding,NCBI,landmark,ACLY


In [7]:
gene_selector = np.array([x in geneinfo2.gene_id.values for x in gene_meta])
print(gene_selector[0:5])
print(sum(gene_selector))

[False  True  True  True  True]
8240


# Step 2: Filter by `DDS_graph` drugs and the CCLE/LINCS cell lines we'll use

Use this to make the sample selector

In [8]:
sample_meta[0:5]

array(['ABY001_A375_XH_X1_B15:A13', 'ABY001_A375_XH_X1_B15:A14',
       'ABY001_A375_XH_X1_B15:A15', 'ABY001_A375_XH_X1_B15:A16',
       'ABY001_A375_XH_X1_B15:A17'], dtype='<U47')

In [9]:
sampinfo.head()

,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_itime,pert_time_unit,cell_mfc_name,pert_mfc_id,det_plate,det_well,rna_plate,rna_well,count_mean,count_cv,qc_f_logp,qc_iqr,qc_slope,pert_id,sample_id,pert_type,cell_iname,qc_pass,dyn_range,inv_level_10,build_name,failure_mode,project_code,cmap_name
0,b11,NaN,20,uL,20 uL,72,72 h,h,VCAP,ERG_11,ERG013_VCAP_72H_X3_B11,O14,ERG013_VCAP_72H_X3,O14,59,18,7.3,11.74,66,ERG_11,ERG013_VCAP_72H_X3_B11:O14,trt_sh,VCAP,0,4.20788,4220.5,NaN,dyn_range,ERG,ERG
1,b10,NaN,1,uL,1 uL,96,96 h,h,U2OS,TRCN0000072237,TAK004_U2OS_96H_X2_B10_DUO52HI53LO,D10,TAK004_U2OS_96H_X1,D10,67,18,5.7,14.98,67,TRCN0000072237,TAK004_U2OS_96H_X2_B10_DUO52HI53LO:D10,ctl_vector,U2OS,0,4.73906,1462,NaN,inv_level_10,TAK,LACZ
2,b12,NaN,0.1,ng/ml,0.1 ng/ml,2,2 h,h,HEPG2,SOD3,CYT001_HEPG2_2H_X2_B12,N12,CYT001_HEPG2_2H_X2,N12,60,17,7.3,11.45,65,SOD3,CYT001_HEPG2_2H_X2_B12:N12,trt_lig,HEPG2,1,6.79642,3038,NaN,NaN,CYT,SOD3
3,b12,NaN,150,ng,150 ng,48,48 h,h,HEK293T,ENTRY00543,HSF038_HEK293T_48H_X2_B12,M01,HSF038_HEK293T_48H_X2,M01,75,15,5.9,9.25,59,ENTRY00543,HSF038_HEK293T_48H_X2_B12:M01,trt_oe,HEK293T,0,23.7971,1642,NaN,inv_level_10,HSF,PDGFRA
4,f3b5,6.66,5.33,uM,6.66 uM,24,24 h,h,A375,BRD-K79781870,DOS043_A375_24H_X1_F3B5_DUO52HI53LO,D17,DOS043_A375_24H_X1,D17,58,26,6.3,15.87,68,BRD-K79781870,DOS043_A375_24H_X1_F3B5_DUO52HI53LO:D17,trt_cp,A375,0,6.78867,1558,NaN,"inv_level_10,qc_iqr",DOS,BRD-K79781870


In [10]:
# only include samples that have passed qc 
sampinfo2 = sampinfo[lambda x: x.qc_pass == "1"]

# grab just the cmp samples 
sampinfo2 = sampinfo2[lambda x: x.pert_type == 'trt_cp']

# grab just the drugs we have in our DDS network 
sampinfo2 = sampinfo2[lambda x: x.cmap_name.isin(drugs)]

# filter to cell lines with CCLE documentation (baseline expr)
sampinfo2 = sampinfo2[lambda x: x.cell_iname.isin(lines)]

print('shape', sampinfo2.shape)

sampinfo2.head()

shape (245235, 30)


,bead_batch,nearest_dose,pert_dose,pert_dose_unit,pert_idose,pert_time,pert_itime,pert_time_unit,cell_mfc_name,pert_mfc_id,det_plate,det_well,rna_plate,rna_well,count_mean,count_cv,qc_f_logp,qc_iqr,qc_slope,pert_id,sample_id,pert_type,cell_iname,qc_pass,dyn_range,inv_level_10,build_name,failure_mode,project_code,cmap_name
7,f1b10,0.002,0.0015,uM,0.002 uM,6,6 h,h,A549,BRD-A61304759,HOG001_A549_6H_X4_F1B10,P11,HOG001_A549_6H_X4,P11,65,20,9.8,9.95,62,BRD-A61304759,HOG001_A549_6H_X4_F1B10:P11,trt_cp,A549,1,12.4715,2301,NaN,NaN,HOG,tanespimycin
49,b1,15,17.1026,uM,15 uM,6,6 h,h,U2OS,BRD-A78377521-001-02-2,PAC005_U2OS_6H_X3_B1_UNI4445L,K10,PAC005_U2OS_6H_X3,K10,82,24,7.5,6.99,65,BRD-A78377521,PAC005_U2OS_6H_X3_B1_UNI4445L:K10,trt_cp,U2OS,1,13.1452,3260,NaN,NaN,PAC,monastrol
51,b1,80,80,uM,80 uM,24,24 h,h,HCC515,BRD-K57080016-001-04-3,CPC006_HCC515_24H_X5_B1_DUO52HI53LO,D04,CPC006_HCC515_24H_X5,D04,78,20,7.8,7.86,59,BRD-K57080016,CPC006_HCC515_24H_X5_B1_DUO52HI53LO:D04,trt_cp,HCC515,1,8.29863,2431.5,NaN,NaN,CPC,selumetinib
53,b10,0.004,0.0045,uM,0.004 uM,24,24 h,h,MCF7,BRD-A84481105,HOG001_MCF7_24H_X1_B10,G10,HOG001_MCF7_24H_X1,G10,50,19,6.8,13.14,66,BRD-A84481105,HOG001_MCF7_24H_X1_B10:G10,trt_cp,MCF7,1,5.13618,3903.5,NaN,NaN,HOG,thioridazine
55,f1b10,0.004,0.0045,uM,0.004 uM,6,6 h,h,MCF7,BRD-K78126613,HOG003_MCF7_6H_X4_F1B10,I21,HOG003_MCF7_6H_X4,I21,76,18,8.2,11.95,62,BRD-K78126613,HOG003_MCF7_6H_X4_F1B10:I21,trt_cp,MCF7,1,8.03753,3641,NaN,NaN,HOG,menadione


In [11]:
mysamples = set(sampinfo2.sample_id.values)
sample_selector = np.array([x in mysamples for x in sample_meta])
print(sum(sample_selector))

245235


# Step 3: Pull observations and genes from the `hdf5` file 

In [12]:
# this can take quite a while 
mydata = h5_data[sample_selector.nonzero()[0], :]

In [13]:
mydata = mydata[:, gene_selector]

In [14]:
mydata = pd.DataFrame(mydata)

In [15]:
gene_cols = pd.Series(gene_meta[gene_selector], name='gene_id').map(geneinfo2[['gene_id', 'myid']].set_index('gene_id')['myid'])

In [16]:
mydata.columns = gene_cols.values

In [17]:
mydata = mydata.assign(sample_id= sample_meta[sample_selector])

In [18]:
mydata.shape

(245235, 8241)

In [19]:
mydata.head()

,ADA,CDH2,AKT3,MED6,ACOT8,ABI1,GNPDA1,ZBTB33,CDH3,TANK,HDAC6,HDAC5,PDCD6IP,PDCD6,BCL2L10,BCL2L11,SH2B3,CDH4,GNE,FRAT1,TROAP,MED16,PIGK,TIMM23,ZNF783,CDH5,CHAF1A,PARP2,PARP3,CDH6,TOM1L1,TOM1,SH2D3A,RANBP9,DNAJB6,MICA,SMC4,GJC1,AP1M2,UBA2,SAE1,DNM1L,CDH8,ABCC9,ABCF2,NR1H3,COX17,SCAMP2,CDH9,DPP3,...,ZBTB40,PAN2,ZBTB5,MFN2,KIF14,JOSD1,CDC25A,PUM3,P2RY14,MAFB,DCLRE1A,ARHGAP25,RBM8A,CDC25B,OXSR1,GFPT2,WDR1,CDC25C,GOLGA5,HS3ST3A1,HS3ST2,HS3ST1,USP15,CDC27,USP3,MVP,TNFSF15,THRAP3,MED12,MED13,CDC34,NUP153,CCS,NR1D2,CLEC2B,RBX1,CDC42,DOP1B,FGFBP1,THOC1,REC8,HNRNPDL,DMTF1,PPP4R1,CDH1,SLC12A6,PTBP3,DGCR2,SCO2,sample_id
0,7.4924,6.56405,9.1052,7.4510,7.70140,6.1691,12.7639,4.51630,3.89000,5.8605,7.757550,6.0334,8.6218,11.46500,5.1312,5.61460,9.9082,5.4493,10.9615,5.13505,7.896000,7.17545,9.1745,9.1660,6.86060,6.5301,7.0301,8.245100,6.09740,4.62080,7.9811,6.939150,4.7799,8.093900,9.1683,7.37600,9.86095,7.088400,6.099300,12.6492,10.0071,11.9641,4.6531,5.1027,9.7246,6.29360,12.32085,6.89545,4.90380,8.6169,...,6.3027,6.92500,5.7850,9.73600,6.2285,9.29705,7.82910,11.0936,3.98790,3.2339,5.22510,5.93670,9.66590,12.1157,8.251101,6.69920,10.3763,5.94940,7.40000,2.6075,4.5212,1.55980,7.1954,7.3965,10.9709,10.2948,5.51160,6.9417,6.0732,8.5739,9.9106,8.054500,6.759900,5.41320,12.8118,10.8286,8.566800,4.1036,4.42080,9.4023,5.23060,10.0028,7.18005,7.6505,1.1633,4.8897,5.5534,6.1668,9.16890,ABY001_A375_XH_X1_B15:A13
1,7.8669,7.67620,9.2727,8.6112,7.33370,5.8269,9.9611,4.88270,4.11210,6.7078,7.237200,5.7254,9.6843,11.83580,5.3833,5.32835,9.7847,5.1489,10.1456,5.33220,7.400600,7.40060,8.3440,9.8791,7.10435,6.4789,7.5310,8.056749,6.57180,5.69455,7.9109,6.613750,5.9759,7.429200,10.4621,6.96025,10.15060,6.710001,7.175299,12.9312,10.1099,11.2434,4.9994,4.6937,10.1845,5.83980,10.89900,7.01595,4.68190,8.7111,...,5.9445,6.28910,5.5770,9.80320,7.2063,8.17410,8.38835,11.4330,6.43275,2.4126,5.09740,4.89450,8.13320,11.6351,8.719300,6.17270,10.1985,6.46470,7.08770,2.6302,4.2215,4.87610,7.4542,7.7907,9.9849,8.9196,5.54835,6.8359,6.1657,8.5109,8.8504,8.538200,6.102800,5.39060,12.9333,10.0504,9.640600,3.7262,5.97550,9.3739,5.77195,10.3324,6.85075,8.0699,1.8844,4.3627,5.8257,5.9020,7.61165,ABY001_A375_XH_X1_B15:A14
2,7.0399,4.93370,8.4097,8.7282,6.88025,5.9476,12.2874,5.00175,5.78490,7.7989,7.630701,6.3948,9.5346,10.50825,5.2283,5.88760,8.3057,6.0251,10.1393,5.05570,7.853900,8.32100,7.2045,9.1501,6.52810,5.0439,7.2617,8.166300,7.00770,4.29610,6.3065,7.993200,5.9494,7.918400,10.6066,7.31960,9.03120,6.665400,7.105000,11.8415,9.5809,11.5554,5.0846,4.7913,9.4279,5.93960,10.07950,6.43070,4.65705,8.0675,...,6.5596,6.79485,6.4838,8.81840,5.4951,9.42710,7.78960,11.1455,4.37490,1.7258,5.19370,5.75950,10.20145,11.5960,9.301400,5.49065,9.7611,5.42625,6.56105,3.0184,3.8900,3.67660,7.4895,7.5628,11.1484,11.2133,5.93725,6.8140,7.0941,9.6764,9.8433,8.434600,5.794700,6.18830,13.0425,9.9283,8.516200,4.2390,4.62015,9.8843,5.17430,11.3330,7.29980,7.9811,0.9265,5.0362,5.7679,7.1669,9.14580,ABY001_A375_XH_X1_B15:A15
3,7.3355,7.82995,8.7718,8.5942,7.36960,5.2347,10.5120,4.87995,4.38045,6.7828,7.526350,6.2044,9.0221,11.66710,5.2433,5.67045,9.9296,5.2615,10.1784,5.30880,7.669799,7.31550,8.2922,10.6585,6.79955,7.5811,7.5017,8.142750,6.57330,5.16920,8.6652,6.759501,5.3115,7.449999,10.4992,7.59480,9.90520,7.231300,5.273200,12.5222,9.8343,11.0152,4.6258,4.6459,9.5237,6.52570,11.38280,7.16060,4.66325,8.4843,...,5.8596,3.61560,5.8022,9.88365,6.3250,8.12420,8.92820,11.3816,4.45100,2.8554,5.33225,4.38970,7.90410,11.7573,8.915800,5.27770,10.5628,6.10360,6.66780,4.0024,4.4177,3.31975,7.0631,7.8776,10.0913,9.4820,5.89030,6.8097,6.5518,8.1379,10.0362,8.495501,6.128600,4.96435,12.1622,11.0559,10.768400,3.6575,4.72820,9.1671,4.59995,11.5465,6.95590,7.7559,2.5469,4.0316,5.6360,6.4808,8.32810,ABY001_A375_XH_X1_B15:A16
4,6.5117,4.89115,9.2922,8.6256,6.08830,5.6700,12.1923,4.88740,4.07565,5.7264,7.928300,6.6864,9.0390,11.52975,4.8198,5.49380,9.7140,5.5293,10.1890,4.94060,8.348000,7.84520,8.4415,10.5552,6.74630,6.

# summarize

In [20]:
sampinfo2.groupby('qc_pass').count()['sample_id']

sampinfo2.groupby('failure_mode').count()['sample_id'].sort_values()

sampinfo2.groupby('pert_dose').count()['sample_id']

sampinfo2.groupby('pert_dose_unit').count()['sample_id'].sort_values()

sampinfo2.groupby('pert_time').count()['sample_id'].sort_values()

pert_time
72        33
2        148
4        277
3       2668
48      5102
6      53406
24    183601
Name: sample_id, dtype: int64

# Step 6: Save data

In [21]:
data = {'pert_expr'         :   mydata, 
        'samp_info'         :   sampinfo2,
        'gene_info'         :   geneinfo2,
        'meta'              :   'LINCS L1000 LVL3 perturbed expression data',
        'source'            :   './LINCS/LVL3/notebooks/make_perturbed_expression.ipynb',
        'creation_date'     :   dt.datetime.now().__str__()}

In [22]:
with open('../../../cell_pertub.pkl', 'wb') as f: 
    pkl.dump(data, f)

In [24]:
data['creation_date']

'2021-06-11 11:12:20.384890'